#Load required Library and module

In [59]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

#Load dataset

In [60]:
sample=pd.read_csv('sample_submission.csv')
sample.head()

,instance_id,music_genre
0,MSC_83537.0,Jazz
1,MSC_22044.0,Jazz
2,MSC_62017.0,Rap
3,MSC_76365.0,Rock
4,MSC_71493.0,Classical


In [61]:
Train_df=pd.read_csv('train.csv')
Test_df=pd.read_csv('test.csv')
Train_df.head()

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,music_genre
0,MSC_83537.0,Estrellitas y Duendes,49.178,0.970522,0.580508,214625.776,0.192107,0.000000,Scale E,0.147134,-14.140,NaN,Major,0.046404,143.78799999999998,Band,0.598965,Jazz
1,MSC_22044.0,Al Norte,59.827,1.009380,0.687542,216232.195,0.265942,0.000031,Scale A,0.174655,-13.716,Male,?,0.039949,?,Band,0.357194,Jazz
2,MSC_62017.0,Yeah! (feat. Lil Jon & Ludacris),89.023,0.020041,0.967948,273314.723,0.857411,0.000000,Scale D,0.042030,-4.995,Female,Major,0.119917,105.01799999999999,Band,0.635525,Rap
3,MSC_76365.0,Can’t You See,55.762,0.010284,0.616287,189189.605,0.970513,0.000220,Scale D,0.124637,-4.262,Both,Major,0.167493,?,Duet,0.983750,Rock
4,MSC_71493.0,"Sonata III (G Moll), BWV 1029: Adagio",45.095,0.858769,0.280645,410136.987,0.114732,0.002753,Scale F Sharp,0.134782,-26.922,Female,Minor,0.039139,112.18299999999999,Band,0.074412,Classical


In [62]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15681 entries, 0 to 15680
Data columns (total 18 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        15681 non-null  object 
 1   track_name         15681 non-null  object 
 2   popularity         15039 non-null  float64
 3   acousticness       15681 non-null  float64
 4   danceability       15125 non-null  float64
 5   duration_ms        15587 non-null  float64
 6   energy             15587 non-null  float64
 7   instrumentalness   15586 non-null  float64
 8   key                15681 non-null  object 
 9   liveness           15681 non-null  float64
 10  loudness           15645 non-null  float64
 11  voice_gender       14916 non-null  object 
 12  mode               15646 non-null  object 
 13  speechiness        15655 non-null  float64
 14  tempo              15681 non-null  object 
 15  musician_category  14321 non-null  object 
 16  valence            150

#Dataset cleaning for Train set

In [63]:
Train_df.drop(columns=['instance_id','track_name'],axis=1,inplace=True)


In [64]:
Train_df.isnull().sum()

popularity            642
acousticness            0
danceability          556
duration_ms            94
energy                 94
instrumentalness       95
key                     0
liveness                0
loudness               36
voice_gender          765
mode                   35
speechiness            26
tempo                   0
musician_category    1360
valence               677
music_genre             0
dtype: int64

In [65]:
Train_df['key'].unique()

array(['Scale E', 'Scale A', 'Scale D', 'Scale F Sharp', 'Scale G Sharp',
       'Scale C Sharp', 'Scale A Sharp', 'Scale F', 'Scale G', 'Scale B',
       'Scale C', 'Scale D Sharp'], dtype=object)

In [66]:
Train_df['voice_gender'].unique()

array([nan, 'Male', 'Female', 'Both'], dtype=object)

In [67]:
Train_df['voice_gender'].replace({'Male':1,'Female':0,'Both':2,np.NaN:3},inplace=True)

In [68]:
Train_df['voice_gender'].unique()

array([3, 1, 0, 2])

In [69]:
Train_df['mode'].unique()

array(['Major', '?', 'Minor', nan], dtype=object)

In [70]:
Train_df['mode'].replace({'Major':1,'Minor':0,'?':2,np.NaN:3},inplace=True)

In [71]:
Train_df['mode'].unique()

array([1, 2, 0, 3])

In [72]:
Train_df['musician_category'].unique()

array(['Band', 'Duet', 'Solo', nan], dtype=object)

In [73]:
Train_df['musician_category'].replace({'Duet':3,'Band':2,'Solo':1,np.NaN:0},inplace=True)

In [74]:
Train_df['musician_category'].unique()

array([2, 3, 1, 0])

In [75]:
Train_df['tempo'].unique()

array(['143.78799999999998', '?', '105.01799999999999', ..., '129.445',
       '123.17', '94.2'], dtype=object)

In [76]:
Train_df['tempo'].replace({'?':100},inplace=True)

In [77]:
Train_df['tempo'] = Train_df['tempo'].astype(float)

In [78]:
Train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15681 entries, 0 to 15680
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   popularity         15039 non-null  float64
 1   acousticness       15681 non-null  float64
 2   danceability       15125 non-null  float64
 3   duration_ms        15587 non-null  float64
 4   energy             15587 non-null  float64
 5   instrumentalness   15586 non-null  float64
 6   key                15681 non-null  object 
 7   liveness           15681 non-null  float64
 8   loudness           15645 non-null  float64
 9   voice_gender       15681 non-null  int64  
 10  mode               15681 non-null  int64  
 11  speechiness        15655 non-null  float64
 12  tempo              15681 non-null  float64
 13  musician_category  15681 non-null  int64  
 14  valence            15004 non-null  float64
 15  music_genre        15681 non-null  object 
dtypes: float64(11), int64(

In [79]:
#Create a DataFrame containing all the non-numeric type features.
Train_category_df=Train_df.select_dtypes(include=['object'])
Train_category_df[Train_category_df.keys()]

,key,music_genre
0,Scale E,Jazz
1,Scale A,Jazz
2,Scale D,Rap
3,Scale D,Rock
4,Scale F Sharp,Classical
...,...,...
15676,Scale B,Classical
15677,Scale F,Jazz
15678,Scale D Sharp,Jazz
15679,Scale F,Rock


In [80]:
Train_dummy_df=pd.get_dummies(Train_df[Train_category_df.keys()[0:1]],drop_first=True,dtype=int)
Train_dummy_df.keys()

Index(['key_Scale A Sharp', 'key_Scale B', 'key_Scale C', 'key_Scale C Sharp',
       'key_Scale D', 'key_Scale D Sharp', 'key_Scale E', 'key_Scale F',
       'key_Scale F Sharp', 'key_Scale G', 'key_Scale G Sharp'],
      dtype='object')

In [81]:
Train_df.drop(columns='key',axis=1,inplace=True)

In [82]:
Train_df2=pd.concat([Train_df,Train_dummy_df],axis=1)
Train_df2.head()

,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,music_genre,key_Scale A Sharp,key_Scale B,key_Scale C,key_Scale C Sharp,key_Scale D,key_Scale D Sharp,key_Scale E,key_Scale F,key_Scale F Sharp,key_Scale G,key_Scale G Sharp
0,49.178,0.970522,0.580508,214625.776,0.192107,0.000000,0.147134,-14.140,3,1,0.046404,143.788,2,0.598965,Jazz,0,0,0,0,0,0,1,0,0,0,0
1,59.827,1.009380,0.687542,216232.195,0.265942,0.000031,0.174655,-13.716,1,2,0.039949,100.000,2,0.357194,Jazz,0,0,0,0,0,0,0,0,0,0,0
2,89.023,0.020041,0.967948,273314.723,0.857411,0.000000,0.042030,-4.995,0,1,0.119917,105.018,2,0.635525,Rap,0,0,0,0,1,0,0,0,0,0,0
3,55.762,0.010284,0.616287,189189.605,0.970513,0.000220,0.124637,-4.262,2,1,0.167493,100.000,3,0.983750,Rock,0,0,0,0,1,0,0,0,0,0,0
4,45.095,0.858769,0.280645,410136.987,0.114732,0.002753,0.134782,-26.922,0,0,0.039139,112.183,2,0.074412,Classical,0,0,0,0,0,0,0,0,1,0,0


In [83]:
Train_df2['popularity'].fillna(Train_df2['popularity'].median(), inplace=True)
Train_df2['danceability'].fillna(Train_df2['danceability'].median(), inplace=True)
Train_df2['duration_ms'].fillna(Train_df2['duration_ms'].median(), inplace=True)
Train_df2['energy'].fillna(Train_df2['energy'].median(), inplace=True)
Train_df2['instrumentalness'].fillna(Train_df2['instrumentalness'].median(), inplace=True)
Train_df2['loudness'].fillna(Train_df2['loudness'].median(), inplace=True)
Train_df2['speechiness'].fillna(Train_df2['speechiness'].median(), inplace=True)
Train_df2['valence'].fillna(Train_df2['valence'].median(), inplace=True)

In [84]:
Train_df2['music_genre'].unique()

array(['Jazz', 'Rap', 'Rock', 'Classical', 'Electronic', 'Country',
       'Hip-Hop'], dtype=object)

In [85]:
Train_df2['music_genre'].replace({'Jazz':1,'Rap':2,'Rock':3,'Classical':4,'Electronic':5,'Country':6,'Hip-Hop':7},inplace=True)

In [86]:
Train_df2.isnull().sum()

popularity           0
acousticness         0
danceability         0
duration_ms          0
energy               0
instrumentalness     0
liveness             0
loudness             0
voice_gender         0
mode                 0
speechiness          0
tempo                0
musician_category    0
valence              0
music_genre          0
key_Scale A Sharp    0
key_Scale B          0
key_Scale C          0
key_Scale C Sharp    0
key_Scale D          0
key_Scale D Sharp    0
key_Scale E          0
key_Scale F          0
key_Scale F Sharp    0
key_Scale G          0
key_Scale G Sharp    0
dtype: int64

#Dataset Cleaning for Test set

In [87]:
Test_df.head()

,instance_id,track_name,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence
0,MSC_70753.0,"Bard, a Man of Lake-town - Extended Version",26.101,0.823765,0.115490,211462.287,0.125325,0.945272,Scale A Sharp,0.120392,-25.541,Both,Minor,0.038657,80.777,Duet,0.041238
1,MSC_24064.0,New Kings,66.325,0.015441,0.592838,249151.238,0.870798,0.000108,Scale E,0.406722,-3.905,Male,?,0.051368,76.00399999999999,Band,0.456309
2,MSC_22731.0,Silver Lining,71.871,0.001006,0.510340,215693.240,0.683077,0.000000,Scale C,0.084608,-6.825,Female,Major,0.034303,144.458,Solo,0.193332
3,MSC_32095.0,A Tale That Wasn't Right,56.151,0.315650,0.338336,307056.179,0.608904,0.000000,Scale A,0.276811,-8.382,Female,Minor,0.036934,118.145,Solo,0.214906
4,MSC_24198.0,Get Ready,43.687,0.038336,0.773904,352802.872,0.955500,0.000007,Scale D,0.301726,-4.733,Both,Major,0.079688,95.012,Duet,0.757347


In [88]:
Test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 17 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        3921 non-null   object 
 1   track_name         3921 non-null   object 
 2   popularity         3755 non-null   float64
 3   acousticness       3921 non-null   float64
 4   danceability       3782 non-null   float64
 5   duration_ms        3900 non-null   float64
 6   energy             3888 non-null   float64
 7   instrumentalness   3898 non-null   float64
 8   key                3921 non-null   object 
 9   liveness           3921 non-null   float64
 10  loudness           3918 non-null   float64
 11  voice_gender       3735 non-null   object 
 12  mode               3907 non-null   object 
 13  speechiness        3914 non-null   float64
 14  tempo              3921 non-null   object 
 15  musician_category  3609 non-null   object 
 16  valence            3730 

In [89]:
Test_df.isnull().sum()

instance_id            0
track_name             0
popularity           166
acousticness           0
danceability         139
duration_ms           21
energy                33
instrumentalness      23
key                    0
liveness               0
loudness               3
voice_gender         186
mode                  14
speechiness            7
tempo                  0
musician_category    312
valence              191
dtype: int64

In [90]:
Test_df.drop(columns=['track_name'],axis=1,inplace=True)

In [91]:
Test_df['voice_gender'].replace({'Male':1,'Female':0,'Both':2,np.NaN:3},inplace=True)
Test_df['mode'].replace({'Major':1,'Minor':0,'?':2,np.NaN:3},inplace=True)
Test_df['musician_category'].replace({'Duet':3,'Band':2,'Solo':1,np.NaN:0},inplace=True)
Test_df['tempo'].replace({'?':100},inplace=True)
Test_df['tempo'] = Test_df['tempo'].astype(float)

In [92]:
Test_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3921 entries, 0 to 3920
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   instance_id        3921 non-null   object 
 1   popularity         3755 non-null   float64
 2   acousticness       3921 non-null   float64
 3   danceability       3782 non-null   float64
 4   duration_ms        3900 non-null   float64
 5   energy             3888 non-null   float64
 6   instrumentalness   3898 non-null   float64
 7   key                3921 non-null   object 
 8   liveness           3921 non-null   float64
 9   loudness           3918 non-null   float64
 10  voice_gender       3921 non-null   int64  
 11  mode               3921 non-null   int64  
 12  speechiness        3914 non-null   float64
 13  tempo              3921 non-null   float64
 14  musician_category  3921 non-null   int64  
 15  valence            3730 non-null   float64
dtypes: float64(11), int64(3)

In [93]:
Test_category_df=Test_df.select_dtypes(include=['object'])
Test_category_df

,instance_id,key
0,MSC_70753.0,Scale A Sharp
1,MSC_24064.0,Scale E
2,MSC_22731.0,Scale C
3,MSC_32095.0,Scale A
4,MSC_24198.0,Scale D
...,...,...
3916,MSC_80955.0,Scale B
3917,MSC_72767.0,Scale E
3918,MSC_40192.0,Scale G
3919,MSC_56067.0,Scale D Sharp


In [94]:
Test_dummy_df=pd.get_dummies(Test_df[Test_category_df.keys()[1:]],drop_first=True,dtype=int)
Test_dummy_df.keys()

Index(['key_Scale A Sharp', 'key_Scale B', 'key_Scale C', 'key_Scale C Sharp',
       'key_Scale D', 'key_Scale D Sharp', 'key_Scale E', 'key_Scale F',
       'key_Scale F Sharp', 'key_Scale G', 'key_Scale G Sharp'],
      dtype='object')

In [95]:
Test_df.drop(columns=Test_category_df.keys()[1:],axis=1,inplace=True)

In [96]:
Test_df2=pd.concat([Test_df,Test_dummy_df],axis=1)
Test_df2.head()

,instance_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,liveness,loudness,voice_gender,mode,speechiness,tempo,musician_category,valence,key_Scale A Sharp,key_Scale B,key_Scale C,key_Scale C Sharp,key_Scale D,key_Scale D Sharp,key_Scale E,key_Scale F,key_Scale F Sharp,key_Scale G,key_Scale G Sharp
0,MSC_70753.0,26.101,0.823765,0.115490,211462.287,0.125325,0.945272,0.120392,-25.541,2,0,0.038657,80.777,3,0.041238,1,0,0,0,0,0,0,0,0,0,0
1,MSC_24064.0,66.325,0.015441,0.592838,249151.238,0.870798,0.000108,0.406722,-3.905,1,2,0.051368,76.004,2,0.456309,0,0,0,0,0,0,1,0,0,0,0
2,MSC_22731.0,71.871,0.001006,0.510340,215693.240,0.683077,0.000000,0.084608,-6.825,0,1,0.034303,144.458,1,0.193332,0,0,1,0,0,0,0,0,0,0,0
3,MSC_32095.0,56.151,0.315650,0.338336,307056.179,0.608904,0.000000,0.276811,-8.382,0,0,0.036934,118.145,1,0.214906,0,0,0,0,0,0,0,0,0,0,0
4,MSC_24198.0,43.687,0.038336,0.773904,352802.872,0.955500,0.000007,0.301726,-4.733,2,1,0.079688,95.012,3,0.757347,0,0,0,0,1,0,0,0,0,0,0


In [97]:
Test_df2['popularity'].fillna(Test_df2['popularity'].median(), inplace=True)
Test_df2['danceability'].fillna(Test_df2['danceability'].median(), inplace=True)
Test_df2['duration_ms'].fillna(Test_df2['duration_ms'].median(), inplace=True)
Test_df2['energy'].fillna(Test_df2['energy'].median(), inplace=True)
Test_df2['instrumentalness'].fillna(Test_df2['instrumentalness'].median(), inplace=True)
Test_df2['loudness'].fillna(Test_df2['loudness'].median(), inplace=True)
Test_df2['speechiness'].fillna(Test_df2['speechiness'].median(), inplace=True)
Test_df2['valence'].fillna(Test_df2['valence'].median(), inplace=True)

In [98]:
Test_df2.isnull().sum()

instance_id          0
popularity           0
acousticness         0
danceability         0
duration_ms          0
energy               0
instrumentalness     0
liveness             0
loudness             0
voice_gender         0
mode                 0
speechiness          0
tempo                0
musician_category    0
valence              0
key_Scale A Sharp    0
key_Scale B          0
key_Scale C          0
key_Scale C Sharp    0
key_Scale D          0
key_Scale D Sharp    0
key_Scale E          0
key_Scale F          0
key_Scale F Sharp    0
key_Scale G          0
key_Scale G Sharp    0
dtype: int64

#Train-Test Split of features and target dataset


In [99]:
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense

In [100]:
features_list=list(Train_df2.columns)
features_list.remove('music_genre')


In [101]:
Train_df2.shape

(15681, 26)

In [102]:
X_train=Train_df2[features_list][:10977] # 70% of dataset taken for train
X_val=Train_df2[features_list][10977:] # 30% of dataset taken for validation
y_train=Train_df2['music_genre'][:10977]
y_val=Train_df2['music_genre'][10977:]
X_test=Test_df2[features_list]


In [103]:
print(X_train.shape)
print(y_train.shape)
print(X_val.shape)
print(y_val.shape)

(10977, 25)
(10977,)
(4704, 25)
(4704,)


In [104]:
X_test.shape

(3921, 25)

In [105]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_val= scale.fit_transform(X_val)

In [106]:
X_test=scale.fit_transform(X_test)


#Build model

In [113]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
classifier = LogisticRegression(random_state =0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_val)

print("LR model accuracy(in %):", metrics.accuracy_score(y_val, y_pred)*100)

print("f1_score\t:", 100*(metrics.f1_score(y_val,y_pred,average='macro')))
print("recall_score\t:",100*(metrics.recall_score(y_val,y_pred,average='macro')))
print("precision_score\t:",100*(metrics.precision_score(y_val,y_pred,average='macro')))


LR model accuracy(in %): 74.21343537414967
f1_score	: 55.469596627494255
recall_score	: 54.22842265557555
precision_score	: 59.66604420298714


In [108]:
y_pred

array([3, 4, 3, ..., 1, 3, 1])

In [109]:
result=[]
for i in range(3921):
    id=Test_df2['instance_id'][i]
    if y_pred[i]==1:
        genre='Jazz'
    elif y_pred[i]==2:
        genre='Rap'
    elif y_pred[i]==3:
        genre='Rock'
    elif y_pred[i]==4:
        genre='Classical'
    elif y_pred[i]==5:
        genre='Electronic'
    elif y_pred[i]==6:
        genre='Country'
    elif y_pred[i]==7:
        genre='Hip-Hop'
    result.append([id,genre])

result_df=pd.DataFrame()
result_df['instance_id']=[i[0] for i in result]
result_df['music_genre']=[i[1] for i in result]
result_df.to_csv("result_LR.csv",index=False)
result_df.head()


,instance_id,music_genre
0,MSC_70753.0,Rock
1,MSC_24064.0,Classical
2,MSC_22731.0,Rock
3,MSC_32095.0,Jazz
4,MSC_24198.0,Rap


#SVM

In [110]:
from sklearn import svm
s = svm.SVC(kernel="linear")
s.fit(X_train,y_train)
y_pred=s.predict(X_val)


print("SVM model accuracy(in %):", metrics.accuracy_score(y_val, y_pred)*100)


SVM model accuracy(in %): 74.6811224489796


In [111]:
print("f1_score\t:", 100*(metrics.f1_score(y_val,y_pred,average='macro')))
print("recall_score\t:",100*(metrics.recall_score(y_val,y_pred,average='macro')))
print("precision_score\t:",100*(metrics.precision_score(y_val,y_pred,average='macro')))


f1_score	: 55.44603697577336
recall_score	: 54.00022302456199
precision_score	: 61.31983996697918


In [112]:
result=[]
for i in range(3921):
    id=Test_df2['instance_id'][i]
    if y_pred[i]==1:
        genre='Jazz'
    elif y_pred[i]==2:
        genre='Rap'
    elif y_pred[i]==3:
        genre='Rock'
    elif y_pred[i]==4:
        genre='Classical'
    elif y_pred[i]==5:
        genre='Electronic'
    elif y_pred[i]==6:
        genre='Country'
    elif y_pred[i]==7:
        genre='Hip-Hop'
    result.append([id,genre])

result_df=pd.DataFrame()
result_df['instance_id']=[i[0] for i in result]
result_df['music_genre']=[i[1] for i in result]
result_df.to_csv("result_SVM.csv",index=False)
result_df.head()


,instance_id,music_genre
0,MSC_70753.0,Rock
1,MSC_24064.0,Classical
2,MSC_22731.0,Rock
3,MSC_32095.0,Electronic
4,MSC_24198.0,Rap
